# Unit test

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [82]:
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
from skforecast.metrics import root_mean_squared_scaled_error
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries


# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y_feature_selection
from skforecast.model_selection.tests.fixtures_model_selection import exog_feature_selection


metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
#metrics = [add_y_train_argument(metric) for metric in metrics]
from skforecast.datasets import fetch_dataset



In [140]:
# data = fetch_dataset(name="items_sales", verbose=False)
# data['day_of_week'] = data.index.dayofweek
# end_train = "2014-07-15 23:59:00"
# initial_train_size = len(data.loc[:end_train, :].copy())
# data_test = data.loc[end_train:, :].copy()
# levels = ["item_1", "item_2", "item_3"]
# exog_features = 'day_of_week'

# metrics = [
#     "mean_absolute_error",
#     "mean_squared_error",
#     mean_absolute_percentage_error,
#     mean_absolute_scaled_error,
#     root_mean_squared_scaled_error,
# ]
# steps = 1
# initial_train_size = 100
# param_grid = {
#     "alpha": np.logspace(-3, 3, 1),
# }
# lags_grid = [3]
# param_grid = list(ParameterGrid(param_grid))

# forecaster = ForecasterAutoregMultiSeries(
#     regressor=Ridge(random_state=678),
#     lags=3,
#     transformer_series=StandardScaler(),
#     transformer_exog=StandardScaler(),
# )

# forecaster = ForecasterAutoregMultiVariate(
#     regressor=Ridge(random_state=678),
#     lags=3,
#     steps=1,
#     level="item_1",
#     transformer_series=StandardScaler(),
#     transformer_exog=StandardScaler(),
# )

# X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
#     forecaster._train_test_split_one_step_ahead(
#         series=data.loc[:, levels],
#         exog=data.loc[:, exog_features] if exog_features else None,
#         initial_train_size=initial_train_size,
#     )
# )

# display(data)
# display(X_train)
# display(X_test)
# display(y_train)
# display(y_test)
# display(X_train_encoding)
# display(X_test_encoding)


# results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
#         forecaster         = forecaster,
#         series             = data.loc[:, levels],
#         exog               = data.loc[:, exog_features] if exog_features else None,
#         param_grid         = param_grid,
#         lags_grid          = lags_grid,
#         metric             = metrics,
#         initial_train_size = initial_train_size,
#         method             = 'one_step_ahead',
#         return_best        = False,
#         verbose            = False,
#         show_progress      = True
#     )

# results_one_step_ahead

In [149]:
# data = fetch_dataset(name="items_sales", verbose=False)
# data['day_of_week'] = data.index.dayofweek
# end_train = "2014-07-15 23:59:00"
# initial_train_size = len(data.loc[:end_train, :].copy())
# levels = ["item_1", "item_2", "item_3"]
# exog_features = ['day_of_week']

# def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
#     forecaster,
# ):

#     metrics = [
#         "mean_absolute_error",
#         "mean_squared_error",
#         mean_absolute_percentage_error,
#         mean_absolute_scaled_error,
#         root_mean_squared_scaled_error,
#     ]
#     steps = 1
#     initial_train_size = 100
#     param_grid = {
#         "alpha": np.logspace(-1, 1, 1),
#     }
#     lags_grid = [3]
#     param_grid = list(ParameterGrid(param_grid))
#     results_backtesting = _evaluate_grid_hyperparameters_multiseries(
#         forecaster         = deepcopy(forecaster),
#         series             = data.loc[:, levels],
#         exog               = data.loc[:, exog_features] if exog_features else None,
#         param_grid         = param_grid,
#         lags_grid          = lags_grid,
#         steps              = steps,
#         refit              = False,
#         metric             = metrics,
#         initial_train_size = initial_train_size,
#         method             = 'backtesting',
#         aggregate_metric   = ["average", "weighted_average", "pooling"],
#         fixed_train_size   = False,
#         return_best        = False,
#         n_jobs             = 'auto',
#         verbose            = False,
#         show_progress      = False
#     )
#     results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
#         forecaster         = deepcopy(forecaster),
#         series             = data.loc[:, levels],
#         exog               = data.loc[:, exog_features] if exog_features else None,
#         param_grid         = param_grid,
#         lags_grid          = lags_grid,
#         metric             = metrics,
#         initial_train_size = initial_train_size,
#         method             = 'one_step_ahead',
#         aggregate_metric   = ["average", "weighted_average", "pooling"],
#         return_best        = False,
#         verbose            = False,
#         show_progress      = False
#     )


#     pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


# regressor = Ridge(random_state=678)
# forecasters = [
#     ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
#     ForecasterAutoregMultiSeries(
#         regressor=regressor,
#         lags=3,
#         transformer_series=None,
#     ),
#     ForecasterAutoregMultiSeries(
#         regressor=regressor,
#         lags=3,
#         transformer_series=StandardScaler(),
#         transformer_exog=StandardScaler()
#     ),
#     ForecasterAutoregMultiVariate(
#         regressor=regressor,
#         level='item_1',
#         lags=3,
#         steps=1,
#         transformer_series=StandardScaler(),
#         transformer_exog=StandardScaler()
#     )

# ]

# for i, forecaster in enumerate(forecasters):
#     print(i)
#     test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
#         forecaster=forecaster
#     )

In [150]:
# from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series
# from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import exog

# series.index = pd.date_range(start='2024-01-01', periods=len(series), freq='D')
# exog.index = pd.date_range(start='2024-01-01', periods=len(exog), freq='D')

# def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
#     forecaster,
# ):

#     metrics = [
#         "mean_absolute_error",
#         "mean_squared_error",
#         mean_absolute_percentage_error,
#         mean_absolute_scaled_error,
#         root_mean_squared_scaled_error,
#     ]
#     steps = 1
#     initial_train_size = 20
#     param_grid = {
#         "alpha": np.logspace(-1, 1, 3),
#     }
#     lags_grid = [3, 7]
#     param_grid = list(ParameterGrid(param_grid))
#     results_backtesting = _evaluate_grid_hyperparameters_multiseries(
#         forecaster         = forecaster,
#         series             = series,
#         exog               = exog,
#         param_grid         = param_grid,
#         lags_grid          = lags_grid,
#         steps              = steps,
#         refit              = False,
#         metric             = metrics,
#         initial_train_size = initial_train_size,
#         method             = 'backtesting',
#         aggregate_metric   = ["average", "weighted_average", "pooling"],
#         fixed_train_size   = False,
#         return_best        = False,
#         n_jobs             = 'auto',
#         verbose            = False,
#         show_progress      = False
#     )
#     results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
#         forecaster         = forecaster,
#         series             = series,
#         exog               = exog,
#         param_grid         = param_grid,
#         lags_grid          = lags_grid,
#         metric             = metrics,
#         initial_train_size = initial_train_size,
#         method             = 'one_step_ahead',
#         aggregate_metric   = ["average", "weighted_average", "pooling"],
#         return_best        = False,
#         verbose            = False,
#         show_progress      = False
#     )


#     pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


# regressor = Ridge(random_state=678)
# forecasters = [
#     ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
#     ForecasterAutoregMultiSeries(
#         regressor=regressor,
#         lags=3,
#         transformer_series=None,
#     ),
#     ForecasterAutoregMultiSeries(
#         regressor=regressor,
#         lags=3,
#         transformer_series=StandardScaler(),
#         transformer_exog=StandardScaler()
#     ),
#     ForecasterAutoregMultiVariate(
#         regressor=regressor,
#         level='l1',
#         lags=3,
#         steps=1,
#         transformer_series=StandardScaler(),
#         transformer_exog=StandardScaler()
#     )

# ]

# for i, forecaster in enumerate(forecasters):
#     print(i)
#     test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
#         forecaster=forecaster
#     )

In [148]:
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast\\skforecast\\model_selection_multiseries\\tests\\fixture_sample_multi_series.joblib")
exog_dict = joblib.load("c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast\\skforecast\\model_selection_multiseries\\tests\\fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 213
    param_grid = {
        "n_estimators": [10, 20],
        "max_depth": [2, 3],
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = LGBMRegressor(random_state=678, verbose=-1)
forecasters = [
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='item_1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

0


KeyboardInterrupt: 

## _calculate_metrics_multiseries vs _calculate_metrics_multiseries_one_step_ahead

In [114]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [124]:
# Data download
# ==============================================================================
series = fetch_dataset(name="items_sales")
exog = pd.DataFrame(
    {
        'day_of_week': series.index.dayofweek
    },
    index = series.index
)
exog = pd.Series(series.index.dayofweek, index=series.index)
end_train = '2014-07-15 23:59:00'
series_train = series.loc[:end_train, :].copy()
series_test  = series.loc[end_train:, :].copy()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


In [130]:
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123, verbose=-1),
                 lags               = 24,
                 encoding           = 'ordinal',
                 #transformer_series = StandardScaler(),
                 #transformer_exog   = StandardScaler(),
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

X, y = forecaster.create_train_X_y(series=series, exog=exog)
X_train = X.loc[X.index <= end_train, :]
y_train = y.loc[y.index <= end_train]
X_test = X.loc[X.index > end_train, :]
y_test = y.loc[y.index > end_train]

forecaster.regressor.fit(X_train, y_train)
pred = forecaster.regressor.predict(X_test)

In [126]:
# Test results _calculate_metrics_multiseries_one_step_ahead and _calculate_metrics_multiseries
# ==============================================================================
# Metrics should be equal when using step=1 in backtesting

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

if forecaster.encoding in ['ordinal', 'ordinal_category']:
    X_train_encoding = forecaster.encoder.inverse_transform(X_train[['_level_skforecast']]).ravel()
    X_test_encoding = forecaster.encoder.inverse_transform(X_test[['_level_skforecast']]).ravel()
elif forecaster.encoding == 'onehot':
    X_train_encoding = forecaster.encoder.inverse_transform(
                            X_train.loc[:, forecaster.encoding_mapping.keys()]
                        ).ravel()
    X_test_encoding = forecaster.encoder.inverse_transform(
                            X_test.loc[:, forecaster.encoding_mapping.keys()]
                        ).ravel()
X_train_encoding = pd.Series(X_train_encoding, index=X_train.index)
X_test_encoding = pd.Series(X_test_encoding, index=X_test.index)

metrics_one_step_ahead = _calculate_metrics_multiseries_one_step_ahead(
    forecaster=forecaster,
    series=series,
    y_true = y_test,
    y_pred = pred,
    y_pred_encoding = X_test_encoding,
    y_train = y_train,
    y_train_encoding = X_train_encoding,
    levels = ['item_1', 'item_2', 'item_3'],
    metrics = metrics,
    add_aggregated_metric = True
)

display(metrics_one_step_ahead)

# Metrics with _calculate_metrics_multiseries
folds = _create_backtesting_folds(
    data = series,
    window_size = 24,
    initial_train_size = len(series_train),
    test_size = 1,
    externally_fitted = False,
    refit = False,
    fixed_train_size = True,
    gap = 0,
    skip_folds = None,
    allow_incomplete_fold = True,
    return_all_indexes = False,
    differentiation = None,
    verbose = False
)
_, predictions = backtesting_forecaster_multiseries(
    series=series,
    exog=exog,
    forecaster=forecaster,
    steps=1,
    metric=metrics,
    initial_train_size = len(series_train),
    refit=False,
    add_aggregated_metric=True,
    show_progress=False
)
metrics_backtesting = _calculate_metrics_multiseries(
    series = series,
    predictions= predictions,
    folds= folds,
    span_index= series.index,
    window_size = 24,
    metrics= metrics,
    levels= ['item_1', 'item_2', 'item_3'],
    add_aggregated_metric = True
)

display(metrics_backtesting)

pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.791801,0.039372,0.518747
1,item_2,2.322449,0.150316,0.985925
2,item_3,3.244195,0.218900,0.876061
3,average,2.119482,0.136196,0.793578
4,weighted_average,2.119482,0.136196,0.793578
5,pooling,2.119482,0.136196,0.838277


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.791801,0.039372,0.518747
1,item_2,2.322449,0.150316,0.985925
2,item_3,3.244195,0.218900,0.876061
3,average,2.119482,0.136196,0.793578
4,weighted_average,2.119482,0.136196,0.793578
5,pooling,2.119482,0.136196,0.838277
